# Quick Start Guide
This tutorial shows how to quickly get started using scarlet to reduce a sample image cube.
In order to run this tutorial you will need either astropy (http://www.astropy.org) or sep (https://github.com/kbarbary/sep) installed to open/create the source catalog and matplotlib (https://matplotlib.org) to display the images

In [ ]:
# Import Packages and setup
import logging

import numpy as np

import scarlet
import scarlet.display

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

## Load and display the sample data

### Load the sample data and source catalog
Loading the source catalog requires astropy. If you don't have astropy installed you'll have to do the source detection yourself. For example, using `sep`:

```python3
import sep
def makeCatalog(img):
    detect = img.mean(axis=0) # simple average for detection
    bkg = sep.Background(detect)
    catalog = sep.extract(detect, 1.5, err=bkg.globalrms)
    bg_rms = np.array([sep.Background(band).globalrms for band in img])
    return catalog, bg_rms
catalog, bg_rms = makeCatalog(images)
```

Otherwise you can just load the "true" catalog:

In [ ]:
# Load the sample images
data = np.load("../data/test_sim/data.npz")
images = data["images"]
filters = data["filters"]

from astropy.table import Table as ApTable
catalog = ApTable.read("../data/test_sim/true_catalog.fits")
bg_rms = np.array([20]*len(images))

### Display a raw image cube
This is an example of how to display an RGB image from an image cube of multiband data. In this case the image uses a $sin^{-1}$ function to normalize the flux and maps i,r,g (filters 3,2,1) $\rightarrow$ RGB.

In [ ]:
# Use Asinh scaling for the images
norm = scarlet.display.Asinh(img=images, Q=50)
# Map i,r,g -> RGB
filter_indices = [3,2,1]
# Convert the image to an RGB image
img_rgb = scarlet.display.img_to_rgb(images, filter_indices=filter_indices, norm=norm)
plt.imshow(img_rgb)
for src in catalog:
    plt.plot(src["x"], src["y"], "rx", mew=2)
plt.show()

## Initialize the sources
Each source must be initialized using `scarlet.Source` or a derived class, in this case `ExtendedSource`, which requires the source to be monotonic and symmetric.

In [ ]:
sources = [scarlet.ExtendedSource((src['y'],src['x']), images, bg_rms) for src in catalog]

## Create and fit the model
The `scarlet.Blend` class contains the machinery to model all of the sources in the blend (provided they have been initialized). In this example the code is set to run for a maximum of 200 iterations, but will end early if the likelihood and all of the constraints converge.

In [ ]:
blend = scarlet.Blend(sources, images, bg_rms=bg_rms)
blend.fit(200)

## View the results

### View the full model
First we load the model for the entire blend, and it's residual. Then we display the model using the same $sin^{-1}$ stretch as the full image and a linear stretch for the residual.

In [ ]:
# Load the model and calculate the residual
model = blend.get_model()
residual = images-model
# Create RGB images
model_rgb = scarlet.display.img_to_rgb(model, filter_indices=filter_indices, norm=norm)
residual_rgb = scarlet.display.img_to_rgb(residual, filter_indices=filter_indices)

# Show the data, model, and residual
fig = plt.figure(figsize=(15,5))
ax = [fig.add_subplot(1,3,n+1) for n in range(3)]
ax[0].imshow(img_rgb)
ax[0].set_title("Data")
ax[1].imshow(model_rgb)
ax[1].set_title("Model")
ax[2].imshow(residual_rgb)
ax[2].set_title("Residual")
for m in range(len(blend.sources)):
    y,x = blend.sources[m].center
    ax[0].text(x, y, m, color="b")
    ax[1].text(x, y, m, color="b")
plt.show()

### View the source models
It can also be useful to view the model for each source. For each source we extract the portion of the image contained in the sources bounding box, the true simulated source flux, and the model of the source, scaled so that all of the images have roughly the same pixel scale.

In [ ]:
def get_true_image(m, catalog, filters):
    """Create the true multiband image for a source
    """
    src = catalog[m]
    img = np.array([catalog[m]["intensity_"+f] for f in filters])
    return img

# We can only show the true values if the input catalog has the true intensity data for the sources
# in other words, if you used SEP to build your catalog you do not have the true data.
if "intensity_"+filters[0] in catalog.colnames:
    has_truth = True
    axes = 3
else:
    has_truth = False
    axes = 2

for m in range(len(blend.sources)):
    # Get the model for a single source
    src = blend.sources[m]
    model = blend.get_model(m=m)[src.bb]
    _rgb = scarlet.display.img_to_rgb(model, filter_indices=filter_indices, norm=norm)
    # Get the patch from the original image
    _img = images[src.bb]
    _img_rgb = scarlet.display.img_to_rgb(_img, filter_indices=filter_indices, norm=norm)
    # Set the figure size
    ratio = src.shape[2]/src.shape[1]
    fig_height = 3*src.shape[1]/20
    fig_width = max(3*fig_height*ratio,2)
    fig = plt.figure(figsize=(fig_width, fig_height))
    # Generate and show the figure
    ax = [fig.add_subplot(1,3,n+1) for n in range(3)]
    ax[0].imshow(_img_rgb)
    ax[0].set_title("Data")
    ax[1].imshow(_rgb)
    ax[1].set_title("model {0}".format(m))
    if has_truth:
        # Get the true image for the same source
        truth = get_true_image(m, catalog, filters)[src.bb]
        true_rgb = scarlet.display.img_to_rgb(truth, filter_indices=filter_indices, norm=norm)
        ax[2].imshow(true_rgb)
        ax[2].set_title("truth")
    plt.show()